In [49]:
import os
import vtk
import nibabel as nib
import numpy as np

from lameg.surf import create_surf_gifti, remove_vertices, combine_surfaces, downsample_multiple_surfaces
from lameg.viz import show_surface

In [24]:
def convert_vtk_to_gifti(fname):
    # Initialize a reader for the VTK file
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(fname)
    reader.Update()

    # Retrieve the output as a polydata object
    polydata = reader.GetOutput()

    # Get the vertices (points)
    points = polydata.GetPoints()
    vertices = [points.GetPoint(i) for i in range(points.GetNumberOfPoints())]

    # Get the faces (polygons)
    faces = []
    for i in range(polydata.GetNumberOfCells()):
        cell = polydata.GetCell(i)
        # Assuming the cell is a polygon, extract its point indices
        if cell.GetCellType() == vtk.VTK_TRIANGLE:
            face = [cell.GetPointId(j) for j in range(cell.GetNumberOfPoints())]
            faces.append(face)
            
    surf = create_surf_gifti(np.array(vertices), np.array(faces))
    return surf

In [25]:
hemis=['lh','rh']
orig_surfs=['OuterSurf','MiddleSurf','InnerSurf']
new_surfs=['pial','0.5','white']
n_surfaces=11
layers = np.linspace(1, 0, n_surfaces)
data_dir='/home/bonaiuto/laminar_baby/data/derivatives/217/6m'

In [26]:
# Convert vtk surfaces to gii
for hemi in hemis:
    for o_idx, orig_surf in enumerate(orig_surfs):
        new_surf=new_surfs[o_idx]
        orig_fname=os.path.join(data_dir, f'T1-iso-skullstripped-rmcere-tissue.{hemi}.{orig_surf}.PhysicalSpace.vtk')
        new_fname=os.path.join(data_dir, f'{hemi}.{new_surf}.gii')
        surf = convert_vtk_to_gifti(orig_fname)
        nib.save(surf, new_fname)

In [28]:
_=show_surface(surf)

Output()

In [30]:
for hemi in hemis:
    white=nib.load(os.path.join(data_dir, f'{hemi}.white.gii'))
    pial=nib.load(os.path.join(data_dir, f'{hemi}.pial.gii'))
    for layer in layers:
        if layer>0 and layer<1:
            new_verts=white.darrays[0].data + layer*(pial.darrays[0].data-white.darrays[0].data)
            new_fname=os.path.join(data_dir, f'{hemi}.{layer:.3f}.gii')
            layer_surf = create_surf_gifti(new_verts, white.darrays[1].data)
            nib.save(layer_surf, new_fname)

In [44]:
for hemi in hemis:
    white=nib.load(os.path.join(data_dir, f'{hemi}.white.gii'))
    pial=nib.load(os.path.join(data_dir, f'{hemi}.pial.gii'))
    dist=np.sqrt(np.sum((white.darrays[0].data-pial.darrays[0].data)**2,-1))
    verts_to_remove=np.where(dist<0.5)[0]
    
    for layer in layers:
        if layer==0:
            layer_name='white'            
        elif layer==1:
            layer_name='pial'
        else:
            layer_name=f'{layer:.3f}'
        surf=nib.load(os.path.join(data_dir, f'{hemi}.{layer_name}.gii'))
        surf_no_deep=remove_vertices(surf, verts_to_remove)
        new_fname=os.path.join(data_dir, f'{hemi}.{layer_name}.nodeep.gii')
        nib.save(surf_no_deep, new_fname)

In [48]:
surfaces_to_process=[]
for layer in layers:
    if layer==0:
        layer_name='white'            
    elif layer==1:
        layer_name='pial'
    else:
        layer_name=f'{layer:.3f}'
    surfaces_to_process.append(layer_name)
    lh_surf=nib.load(os.path.join(data_dir, f'lh.{layer_name}.nodeep.gii'))
    rh_surf=nib.load(os.path.join(data_dir, f'rh.{layer_name}.nodeep.gii'))
    combined_fname=os.path.join(data_dir, f'{layer_name}.gii')
    combined = combine_surfaces([lh_surf, rh_surf])
    nib.save(combined, combined_fname)

In [50]:
ds_factor=0.1

in_surfs = []
for surface_name in surfaces_to_process:
    in_surf_fname = os.path.join(data_dir, f'{surface_name}.gii')
    in_surf = nib.load(in_surf_fname)
    in_surfs.append(in_surf)

# Downsample multiple surfaces
out_surfs = downsample_multiple_surfaces(in_surfs, ds_factor)
for surface_name, out_surf in zip(surfaces_to_process, out_surfs):
    out_surf_path = os.path.join(data_dir, f'{surface_name}.ds.gii')
    nib.save(out_surf, out_surf_path)

100.0% of the vertices in the decimated surface belong to the original surface.


In [54]:
orientation='link_vector'
fix_orientation=True

orientations = compute_dipole_orientations(
    orientation,
    surfaces_to_process,
    data_dir,
    fixed=fix_orientation
)

base_fname = f'ds.{orientation}'
if fix_orientation:
    base_fname = f'{base_fname}.fixed'
for l_idx, layer_name in enumerate(surfaces_to_process):
    in_surf_path = os.path.join(data_dir, f'{layer_name}.ds.gii')
    surf = nib.load(in_surf_path)

    # Set these link vectors as the normals for the downsampled surface
    ori_array=nib.gifti.GiftiDataArray(data=orientations[l_idx, :, :],
                                       intent=nib.nifti1.intent_codes['NIFTI_INTENT_VECTOR'])
    surf.add_gifti_data_array(ori_array)

    # Save the modified downsampled surface with link vectors as normals
    out_surf_path = os.path.join(data_dir, f'{layer_name}.{base_fname}.gii')
    nib.save(surf, out_surf_path)

In [56]:
out_fname='multilayer.11.ds.link_vector.fixed.gii'
all_surfs = []
for layer_name in surfaces_to_process:
    surf_path = os.path.join(data_dir, f'{layer_name}.{base_fname}.gii')
    surf = nib.load(surf_path)
    all_surfs.append(surf)

combined = combine_surfaces(all_surfs)
nib.save(combined, os.path.join(data_dir, out_fname))

In [57]:
_=show_surface(combined)

Output()